In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [83]:
df = pd.read_csv('/content/diabetes.csv')

In [84]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [85]:
df.shape

(768, 9)

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [87]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [88]:

x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [89]:
from sklearn.model_selection import train_test_split

In [90]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [91]:
# 1. how to select appropriate optimizer
# 2. No. of nodes in a layer
# 3. how to select no. of layers
# 4. all in one model

In [47]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.1 MB/s eta 0:00:00


In [92]:
import kerastuner as kt

Optimization function

In [151]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model(hp):
    model = Sequential()

    # First dense layer
    model.add(Dense(32, activation='relu', input_dim=8))
    #model.add(Dense(64, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = hp.Choice('optimizer', ['Adam','sgd','rmsprop','adadelta'])

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


In [152]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy', # tell kya krana chahte ho model se
    max_trials=5
)

Reloading Tuner from ./untitled_project/tuner0.json


In [153]:

tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))


In [154]:

tuner.get_best_hyperparameters()[0].values

{'optimizer': 'sgd', 'units': 112}

In [155]:
best_hps = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hps)  # Rebuild with fixed params


In [156]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5859 - loss: 12.0985 - val_accuracy: 0.6494 - val_loss: 0.8919
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6082 - loss: 0.7248 - val_accuracy: 0.5909 - val_loss: 0.6959
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6614 - loss: 0.6745 - val_accuracy: 0.6623 - val_loss: 0.9302
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6581 - loss: 0.6828 - val_accuracy: 0.5974 - val_loss: 0.6718
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6649 - loss: 0.6627 - val_accuracy: 0.3636 - val_loss: 1.0359
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5760 - loss: 0.7461 - val_accuracy: 0.4675 - val_loss: 0.7771
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6823 - loss: 0.6630 - val_accuracy: 0.6948 - val_loss: 0.6392
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6689 - loss: 0.6576 - val_accuracy: 0.6

Number of epochs

In [157]:

def build_model(hp):
    model = Sequential()

    units = hp.Int('units',min_value = 8,max_value = 128,step=8)

    # First dense layer
    model.add(Dense(units = units, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model



In [158]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy', # tell kya krana chahte ho model se
    max_trials=5,
    directory = 'mydir',
    project_name ='Ayush3'
)

In [159]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.6948052048683167

Best val_accuracy So Far: 0.6948052048683167
Total elapsed time: 00h 00m 22s


In [160]:
tuner.get_best_hyperparameters()[0].values

{'units': 72}

In [161]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [162]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 72)             │           648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            73 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 721 (2.82 KB)

 Trainable params: 721 (2.82 KB)

 Non-trainable params: 0 (0.00 B)

In [163]:
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5890 - loss: 0.8917 - val_accuracy: 0.7013 - val_loss: 0.7378
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6788 - loss: 0.7272 - val_accuracy: 0.6688 - val_loss: 0.7923
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7079 - loss: 0.6157 - val_accuracy: 0.6753 - val_loss: 0.8188
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6971 - loss: 0.6146 - val_accuracy: 0.6623 - val_loss: 0.6931
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7234 - loss: 0.5809 - val_accuracy: 0.6688 - val_loss: 0.6943
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7093 - loss: 0.6332 - val_accuracy: 0.6429 - val_loss: 0.7155
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6887 - loss: 0.6503 - val_accuracy: 0.7338 - val_loss: 0.6331
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6873 - loss: 0.6214 - val_accuracy: 0.68

How to select no. of Layer

In [164]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(64,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(32,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [165]:

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy', # tell kya krana chahte ho model se
    max_trials=5,
    directory = 'mydir',
    project_name ='Ayush4'
)

In [166]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 06s]
val_accuracy: 0.7077922224998474

Best val_accuracy So Far: 0.7077922224998474
Total elapsed time: 00h 00m 27s


In [167]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7}

In [168]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [169]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.6269 - loss: 0.6586 - val_accuracy: 0.6429 - val_loss: 0.6467
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6322 - loss: 0.6483 - val_accuracy: 0.6494 - val_loss: 0.6375
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6619 - loss: 0.6071 - val_accuracy: 0.4610 - val_loss: 0.7447
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6612 - loss: 0.6223 - val_accuracy: 0.7273 - val_loss: 0.6168
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6957 - loss: 0.5859 - val_accuracy: 0.5909 - val_loss: 0.6591
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6827 - loss: 0.6018 - val_accuracy: 0.7273 - val_loss: 0.5994
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7247 - loss: 0.5623 - val_accuracy: 0.6818 - val_loss: 0.6157
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6965 - loss: 0.5724 - val_accuracy: 0.70